## Classify Reviews

In [1]:
######importing libraries for data manipulation#######
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gzip
import math
from PIL import Image
import requests
from io import BytesIO
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import Perceptron

In [30]:
###############Function definitions###########

#########Function definitions for separating related feature########
def change_vals_new_col(s,value,new_cols):
    
    if(value.get(s) != None):
        if((type(value[s]) == float) and np.isnan(value[s])):
            new_cols[s] = np.nan
        else:
            new_cols.get(s).append(value[s])
    else:
        new_cols.get(s).append(np.nan)
    

def generate_new_cols(related):
    
    new_cols = {'also_bought':[], 'also_viewed':[],'bought_together':[],'buy_after_viewing':[]}
    
    for key,value in related.items():
        if((type(value) == float) and np.isnan(value)):
            
            new_cols['also_bought'].append(np.nan)
            new_cols['also_viewed'].append(np.nan)
            new_cols['bought_together'].append(np.nan)
            new_cols['buy_after_viewing'].append(np.nan)
        else:
            change_vals_new_col('also_bought',value,new_cols)
            change_vals_new_col('also_viewed',value,new_cols)
            change_vals_new_col('bought_together',value,new_cols)
            change_vals_new_col('buy_after_viewing',value,new_cols)
            
        
    return new_cols
#####Function definitions for separating related feature ends####

def plot_related_prods(related,which,final_metadata):
    
    if(related == None):
        print('People who'+str(related)+'this product did not buy any other product:')
        return
    else:
        #print(np.array(related) in final_metadata.index)
        tot = 0
        for idx in related:
            if(idx in final_metadata.index):
                tot += 1
        print(tot)
        tot = round(tot/2)
        print('final',tot)
        f, axes = plt.subplots(tot,tot,figsize=(4,4),dpi=300)
        f.suptitle('People also '+str(which))
        for i in range(0,tot):
            for j in range(0,tot):
                curr_asin = related[i+j]
                if((curr_asin in final_metadata.index) == True):
                    curr_url = final_metadata.loc[curr_asin]['imUrl']
                    curr_title = final_metadata.loc[curr_asin]['title']
                    curr_title = curr_title[0:30]
                    response = requests.get(curr_url)
                    img = Image.open(BytesIO(response.content))
                    axes[i,j].imshow(img)
                    axes[i,j].get_xaxis().set_ticks([])
                    axes[i,j].get_yaxis().set_ticks([])
                    plt.axis('off')
                    axes[i,j].set_title(curr_title,size=3)
        plt.show()            

def Show_related_products(meta_data_row,final_metadata):
    
    #print(meta_data_row)
    curr_url = meta_data_row['imUrl']
    #curr_prod_id = meta_data_row['asin']
    title = meta_data_row['title']
    
    print('The current product is:',title)
    response = requests.get(curr_url)
    img = Image.open(BytesIO(response.content))
    plt.imshow(img)
    plt.show()
    
    ####People who bought this product also bought####
    also_bought = meta_data_row['also_bought']
    if((type(also_bought) == float) and np.isnan(also_bought)):
        also_bought = None
    else:
        if(len(also_bought) > 9):
            also_bought = also_bought[0:9]
    plot_related_prods(also_bought,'bought',final_metadata)
    
    ####People who bought this product also viewed####
    also_viewed = meta_data_row['also_viewed']
    if((type(also_viewed) == float) and np.isnan(also_viewed)):
        also_viewed = None
    else:
        if(len(also_viewed) > 9):
            also_viewed = also_viewed[0:9]
    plot_related_prods(also_viewed,'viewed',final_metadata)

def Naive_Bayes(X,Y,which,avg='binary'):
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    pred_train = clf.predict(X_train)
    print('Train accuracy score of ',which,' NB is: ',accuracy_score(y_train, pred_train, normalize=True))
    print('Train F-1 ',which,' NB is: ',f1_score(y_train, pred_train,average=avg))
    pred_test = clf.predict(X_test)
    print('Test accuracy score of ',which,' NB is: ',accuracy_score(y_test, pred_test, normalize=True))
    print('Test F-1 ',which,' NB is: ',f1_score(y_test, pred_test,average=avg))
    
def Perceptron_classifier(X,Y,which,weights=None,avg='binary'):
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
    
    clf = Perceptron(max_iter=200,class_weight=weights)
    clf.fit(X_train,y_train)
    pred_train = clf.predict(X_train)
    
    print('Train accuracy score of ',which,' Perceptron: ',accuracy_score(y_train, pred_train, normalize=True))
    print('Train F-1 ',which,' Perceptron is: ',f1_score(y_train, pred_train,average=avg))
    pred_test = clf.predict(X_test)
    print('Test accuracy score of ',which,' Perceptron is: ',accuracy_score(y_test, pred_test, normalize=True))
    print('Test F-1 ',which,' Perceptron is: ',f1_score(y_test, pred_test,average=avg))
    
    
        
    

### Read Data In

In [3]:
#####Reading Data#######
#####This is a smaller data for initial data exploration and model testing#####
######The data is about Heal and Personal care Products on Amazon##########

def parse(path):
    g = open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

data_path = '/Users/ruchinpatel/Desktop/USC_EVERYTHING/SPRINGBOARD/CAPSTONE/Health_and_Personal_Care_5.json'
metadata_path = '/Users/ruchinpatel/Desktop/USC_EVERYTHING/SPRINGBOARD/CAPSTONE/meta_Health_and_Personal_Care.json'

data = getDF(data_path)
metadata = getDF(metadata_path)

* As seen from the table above the field related has two sub values $\textbf{also_bought}$ and $\textbf{also_viewed}$ and as a result of this we need to make two new columns for it.

In [4]:
########## Generating seperate columns for related feature######
related = metadata['related'].to_dict()
newly_created_columns = pd.DataFrame(generate_new_cols(related))

In [5]:
#########Final Metadata dataframe###########
final_metadata = pd.concat([metadata,newly_created_columns],axis = 1)
final_metadata = final_metadata.drop('related',axis=1)
final_metadata = final_metadata.set_index('asin')

######converting the dates to date time format#####
data['unixReviewTime'] = pd.to_datetime(data['unixReviewTime'],unit='s')
data['reviewTime'] = pd.to_datetime(data['reviewTime'])

In [6]:
data.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,ALC5GH8CAMAI7,159985130X,AnnN,"[1, 1]",This is a great little gadget to have around. ...,5.0,Handy little gadget,2011-01-05,2011-01-05
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""","[1, 1]",I would recommend this for a travel magnifier ...,4.0,Small & may need to encourage battery,2012-02-18,2012-02-18
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""","[75, 77]",What I liked was the quality of the lens and t...,4.0,Very good but not great,2010-06-08,2010-06-08
3,A1XZUG7DFXXOS4,159985130X,Cat lover,"[56, 60]",Love the Great point light pocket magnifier! ...,4.0,great addition to your purse,2008-02-08,2008-02-08
4,A1MS3M7M7AM13X,159985130X,Cricketoes,"[1, 1]",This is very nice. You pull out on the magnifi...,5.0,Very nice and convenient.,2011-08-16,2011-08-16


In [7]:
final_metadata.head(5)

,description,title,imUrl,salesRank,categories,price,brand,also_bought,also_viewed,bought_together,buy_after_viewing
asin,,,,,,,,,,,
0077614992,This is an example product description.,Principles of Mgmt + Oper-CSUF Custom C,http://ecx.images-amazon.com/images/I/51G%2BRq...,{'Health & Personal Care': 168429},[[Health & Personal Care]],NaN,NaN,"[0471730726, 0132834871, 0471391905, B00000JZK...","[0073525057, 1133227295, 0324628676, 0073523224]",NaN,NaN
0615208479,By now we all know the benefits of exercise fo...,Brain Fitness Exercises Software,http://ecx.images-amazon.com/images/I/41kbZB04...,{'Health & Personal Care': 1346973},"[[Health & Personal Care, Personal Care]]",NaN,NaN,NaN,NaN,NaN,NaN
0615269990,What's wrong with your patient?Do all the symp...,Occam's Razor,http://ecx.images-amazon.com/images/I/51fH-ABe...,{'Toys & Games': 110575},"[[Health & Personal Care, Personal Care, Shavi...",34.99,NaN,"[1935660152, 0071743979, 0071831428, 032308787...","[1594741476, B0069628EU, B009RTGX2Y, B000IQHSL...",NaN,NaN
0615315860,NaN,101 BlenderBottle Recipes Quick and Easy,http://ecx.images-amazon.com/images/I/21zOQu2Q...,{'Health & Personal Care': 254068},[[Health & Personal Care]],NaN,NaN,"[B006VT9RBM, B0010JLMO8, B001CXC69C, B0064QSHX...",[B0018G4ZEW],NaN,NaN
0615406394,This is an example product description.,"Aphrodite Reborn - Women's Stories of Hope, Co...",http://ecx.images-amazon.com/images/I/51rJLgsi...,{'Health & Personal Care': 377936},[[Health & Personal Care]],NaN,NaN,"[0966035232, 1421407205]",NaN,NaN,NaN


In [8]:
reviews_data = data[['reviewText','overall']]
reviews_data_binary = data[['reviewText','overall']]
replace = {1:0,2:0,3:0,4:1,5:1}
reviews_data['overall_binary'] = reviews_data[['overall']].replace(to_replace=replace,value=None)

#print(reviews_data.shape)
#print(reviews_data_binary.shape)
#print(np.unique(reviews_data_binary['overall']))
#reviews_data[['overall','overall_binary']].head(10)

/Users/ruchinpatel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [11]:
print('Total ratings of all classes')
print()
print()
ratings_count = reviews_data.groupby(by='overall').count()
ratings_count['reviewTextPercent'] = ratings_count['reviewText']*100/np.sum(ratings_count['reviewText'])
ratings_count['classWeights'] = 20/ratings_count['reviewTextPercent']
print(ratings_count[['reviewText','reviewTextPercent','classWeights']])

Total ratings of all classes


         reviewText  reviewTextPercent  classWeights
overall                                             
1.0           16546           4.777179      4.186571
2.0           16754           4.837233      4.134595
3.0           33254           9.601132      2.083088
4.0           68168          19.681541      1.016181
5.0          211633          61.102915      0.327317


In [12]:
#####Tokenizing our text data####
count_vect = CountVectorizer(analyzer = 'word',stop_words = 'english',min_df=0.01,binary=False)
review_text_tokenized = count_vect.fit_transform(reviews_data['reviewText'])

In [13]:
print(review_text_tokenized.shape)
review_text_tokenized = pd.DataFrame(review_text_tokenized.toarray())
ratings_multi = reviews_data['overall']
ratings_binary = reviews_data['overall_binary']

(346355, 625)


In [14]:
#####Naive Bayes Unweighted all 5 classes########
Naive_Bayes(review_text_tokenized,ratings_multi,'Multinomial',avg='weighted')

Train accuracy score of  Multinomial  NB is:  0.6211076785379163
Train F-1  Multinomial  NB is:  0.5864571245079429
Test accuracy score of  Multinomial  NB is:  0.6178198668995684
Test F-1  Multinomial  NB is:  0.5825939747537663


In [15]:
#####Naive Bayes Unweighted binary########
Naive_Bayes(review_text_tokenized,ratings_binary,'Binary',avg='binary')

Train accuracy score of  Binary  NB is:  0.8247101961859942
Train F-1  Binary  NB is:  0.8947370702359693
Test accuracy score of  Binary  NB is:  0.823894559050685
Test F-1  Binary  NB is:  0.8941674546921495


In [31]:
######Perceptron multi-class######
Perceptron_classifier(review_text_tokenized,ratings_multi,'Multiclass',avg='weighted')

Train accuracy score of  Multiclass  Perceptron:  0.5261148243853849
Train F-1  Multiclass  Perceptron is:  0.5271116834842609
Test accuracy score of  Multiclass  Perceptron is:  0.5216035570440732
Test F-1  Multiclass  Perceptron is:  0.5222687548203955


In [32]:
######Perceptron binary######
Perceptron_classifier(review_text_tokenized,ratings_binary,'Binary')

Train accuracy score of  Binary  Perceptron:  0.7853972080668679
Train F-1  Binary  Perceptron is:  0.8668101681475988
Test accuracy score of  Binary  Perceptron is:  0.7855668317189012
Test F-1  Binary  Perceptron is:  0.8669187213302751


In [33]:
######Perceptron multi-class,balanced weights for every class######
Perceptron_classifier(review_text_tokenized,ratings_binary,'Binary','balanced',avg='weighted')

Train accuracy score of  Binary  Perceptron:  0.7294105758542536
Train F-1  Binary  Perceptron is:  0.7512862546245971
Test accuracy score of  Binary  Perceptron is:  0.729295087410316
Test F-1  Binary  Perceptron is:  0.7514480162285552
